In [1]:
import os
import sys
os.chdir('..')
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
print(os.getcwd())

/home/admin/coref_research/gpt-coref-met-1/src


In [3]:
import dtale
import pandas as pd
from collections import defaultdict
from omegaconf import OmegaConf
import jsonlines

from utils.get_processed_dataset import get_processed_dataset
from utils.utils import get_coref_docs, get_major_entities
from configs.config import PRONOUNS_GROUPS
from utils.analysis_utils import *

ModuleNotFoundError: No module named 'src'

In [ ]:
def load_config(config_arg,model_name,run):
    dataset_config = OmegaConf.load(dataset_yaml)
    experiment_config = OmegaConf.load(config_arg)
    experiment_config["model"] = model_name
    experiment_config["run"] = run
    return dataset_config, experiment_config

In [ ]:
models = ["gpt","gpt35","haiku","cl-34b-i","llama3","mst-7b-i"]

In [ ]:
config_arg = "conf/final_table_configs/direct/fantasy.yaml"
model_name = "gpt"
run = "run5"
dataset_config, experiment_config = load_config(config_arg,model_name,run)

doc_dataset = experiment_config["dataset"]
doc_split = experiment_config["split"]
doc_addr = dataset_config[doc_dataset][f"{doc_split}_file"]
doc_tsv_addr = dataset_config[doc_dataset]["tsv"]
is_sota = experiment_config.get("is_sota", False) 
sota_str = "_sota" if is_sota else ""
doc_me = dataset_config[doc_dataset][f"{doc_split}_me{sota_str}"]

head = experiment_config.get("head", False)

## Get the docs and train docs
print("Doc addr: ", doc_addr)
docs = get_coref_docs(doc_addr)
docs_processed = get_processed_dataset(docs,doc_tsv_addr,head=head)
major_entities = get_major_entities(doc_me)

litbank_config= {
                    "name": "LitBank",
                    "cluster_threshold": 1,
                    "canonical_cluster_threshold": 1,
                    "cross_val_split": 0,
                    "metrics":['MUC', 'Bcub', 'CEAFE']
                }

jsonl_file = experiment_config["paths"]["result_destination"]
print("Results jsonlines: ",jsonl_file)
file_name = os.path.splitext(jsonl_file)[0]
xlsx_file = f"{file_name}.xlsx"
txt_file = f"{file_name}_avg.txt"

sys_output = {}
with jsonlines.open(jsonl_file, mode='r') as reader:
    for line in reader:
        if line.get("golden_clusters", None) is None:
            line["golden_clusters"] = line["clusters"]
        sys_output[line['doc_key']] = line


Doc addr:  ../data/raw_data/fantasy/longformer_speaker/test.4096.jsonlines


/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



148_The_Lord’s_Animals_and_the_Devil’s_0
111_The_Skilful_Huntsman_0
198_Maid_Maleen_0
059_Frederick_and_Catherine_0
144_The_Donkey_0
074_The_Fox_and_His_Cousin_0
031_The_Girl_Without_Hands_0
130_One-eye,_Two-eyes,_and_Three-eyes_0
045_Thumbling_as_Journeyman_0
138_Knoist_and_his_Three_Sons_0
001_The_Frog-King,_or_Iron_Henry_0
008_The_Wonderful_Musician_0
109_The_Shroud_0
042_The_Godfather_0
039_The_Elves_0
156_Odds_And_Ends_0
136_Iron_John_0
114_The_Cunning_Little_Tailor_0
145_The_Ungrateful_Son_0
191_The_Sea-Hare_0
Results jsonlines:  ../results/final1/gpt/direct/fantasy/test/full/desc/result.jsonl


In [ ]:
need_to_sort = ['predicted_clusters', 'golden_clusters', 'predicted_clusters_head', 'golden_clusters_head']
for key in need_to_sort:
    for doc_key in sys_output:
        if key in sys_output[doc_key] and sys_output[doc_key][key] is not None:
            for cluster_ind,cluster in enumerate(sys_output[doc_key][key]):
                sys_output[doc_key][key][cluster_ind] = sorted(cluster)

In [ ]:
for doc_key in sys_output:
    print(sys_output[doc_key].keys())

dict_keys(['doc_key', 'predicted_clusters', 'golden_clusters', 'predicted_clusters_head', 'golden_clusters_head', 'modified_candidates', 'added_candidates', 'pattern_failed_candidates'])
dict_keys(['doc_key', 'predicted_clusters', 'golden_clusters', 'predicted_clusters_head', 'golden_clusters_head', 'modified_candidates', 'added_candidates', 'pattern_failed_candidates'])
dict_keys(['doc_key', 'predicted_clusters', 'golden_clusters', 'predicted_clusters_head', 'golden_clusters_head', 'modified_candidates', 'added_candidates', 'pattern_failed_candidates'])
dict_keys(['doc_key', 'predicted_clusters', 'golden_clusters', 'predicted_clusters_head', 'golden_clusters_head', 'modified_candidates', 'added_candidates', 'pattern_failed_candidates'])
dict_keys(['doc_key', 'predicted_clusters', 'golden_clusters', 'predicted_clusters_head', 'golden_clusters_head', 'modified_candidates', 'added_candidates', 'pattern_failed_candidates'])
dict_keys(['doc_key', 'predicted_clusters', 'golden_clusters', 'p

In [ ]:
if head == False:
    print("Reached here")
    get_tbound_equivalent(sys_output, docs_processed)
    original_index_list_key = "mentions_vs_tbound"
else:
    print("Thala")
    original_index_list_key = "mentions_vs_headbound"
    get_tbound_equivalent_head(sys_output, docs_processed)

sys_output = add_keys_to_output(sys_output)

Reached here
0 [[16, 16], [45, 45], [81, 81], [103, 103], [144, 144], [158, 158], [162, 162], [171, 171], [193, 193], [225, 225], [231, 231], [268, 268], [298, 298], [329, 329], [358, 358], [366, 366], [370, 370], [374, 374], [382, 382], [399, 399], [441, 441], [446, 446], [457, 457], [473, 473], [482, 482], [506, 506], [524, 524], [546, 546], [571, 571], [596, 596], [612, 612], [627, 627], [633, 633], [678, 678], [763, 763], [765, 765], [861, 861], [913, 913], [963, 963], [979, 979], [1089, 1089], [1091, 1091], [1094, 1094], [1100, 1100], [1111, 1111], [1152, 1152], [1246, 1246], [1264, 1264], [1273, 1273], [1276, 1276], [1360, 1360], [1373, 1373], [1391, 1391], [1417, 1417], [1449, 1449], [1454, 1454], [1456, 1456], [1481, 1481], [1487, 1487], [1498, 1498], [1505, 1505], [1534, 1534], [1553, 1553], [1584, 1584], [1638, 1638], [1655, 1655], [1658, 1658], [1677, 1677], [1733, 1733], [1745, 1745], [1770, 1770], [1800, 1800], [1850, 1850], [1896, 1896], [1960, 1960], [2009, 2009], [2031,

In [ ]:
### Creating the master error table
decision_dict = defaultdict(list)
decision_dict.update(
{
    "doc_key":[],
    "broad_mention_type":[],
    "nested_mention": [], 
    
    "mention_start": [],
    "mention_end": [],
    "mention_start_stbound": [],
    "mention_end_stbound": [],
    "mention_str": [],
    "mapped_entity_ind":[],
    "mapped_entity_name": [],
    
    "golden_mention":[],
    "major_mention": [],
    "mention_ind": [],
    "entity_ind":[],
    "entity_name": [],
    "category":[],
    "type": [],
    "antecedent_dist_gt": [],
    "next_dist_gt": [],
    "nearest_dist_gt": [],
    "antecedent_dist_pred": [],
    # "antecedent_dist_pred_filt": [],
    
    "correct":[],
    "correct_major":[],
    "detailed_error_type":[],  
    
    "mapped_antecedent_dist_gt": [],
    "mapped_antecedent_dist_pred":[],
    # "mapped_antecedent_dist_pred_filt": [],
    
    ## Can be added later
    # "antecedent_dist", "mapped_antecedent_dist"
    
    ## Score is present as a part of code.
})

In [ ]:
def process_failed_mentions(sys_output, decision_dict):
    ### First let us look into processing failed mentions i.e the 7th cluster of the predicted mentions
    for document in sys_output.keys():
        if len(sys_output[document]["predicted_clusters"]) < len(sys_output[document]["golden_clusters"]) + 1:
            continue 
        failed_mentions = sys_output[document]["predicted_clusters"][-1]
        major_entity_names = major_entities[document]["entity_name"] + ["Others"]
        for mention in failed_mentions:
            decision_dict["doc_key"].append(document)
            decision_dict["broad_mention_type"].append("Failed Mention")
            failed_mention_ind = sys_output[document]["predicted_mentions"].index(mention)
            decision_dict["nested_mention"].append(sys_output[document]["predicted_mentions_nested"][failed_mention_ind]) ## Checking in prediction because?
            
            mention_index_in_predicted = sys_output[document]["predicted_mentions"].index(mention)
            decision_dict["mention_start"].append(mention[0])
            decision_dict["mention_end"].append(mention[1])
            decision_dict["mention_start_stbound"].append(sys_output[document]["predicted_mentions_stbound"][mention_index_in_predicted][0])
            decision_dict["mention_end_stbound"].append(sys_output[document]["predicted_mentions_stbound"][mention_index_in_predicted][1])
            decision_dict["mention_str"].append(get_mention_str_tbound(mention, docs_processed[document]))
            decision_dict["mapped_entity_ind"].append(-1)
            decision_dict["mapped_entity_name"].append("None")
            
            is_golden_mention = 1 if mention in sys_output[document]["gt_mentions"] else 0
            decision_dict["golden_mention"].append(is_golden_mention)
            if is_golden_mention:
                mention_ind_gold_orig = docs_processed[document][original_index_list_key].index(mention)
                entity_ind = sys_output[document]["gt_mentions_vs_clusters"][tuple(mention)]
                if entity_ind < len(major_entity_names) - 1:
                    decision_dict["major_mention"].append(True)
                else:
                    decision_dict["major_mention"].append(False)
                decision_dict["mention_ind"].append(mention_ind_gold_orig)
                decision_dict["entity_ind"].append(entity_ind)
                decision_dict["entity_name"].append(major_entity_names[entity_ind])
                if doc_tsv_addr is not None:
                    decision_dict["category"].append(docs_processed[document]["mentions_vs_mentionctgry"][mention_ind_gold_orig])
                    decision_dict["type"].append(docs_processed[document]["mentions_vs_mentiontype"][mention_ind_gold_orig])     
                else:
                    if decision_dict["mention_str"][-1].lower().strip() in PRONOUNS_GROUPS:
                        decision_dict["category"].append("PRON")
                    else:
                        decision_dict["category"].append("NOM")
                    decision_dict["type"].append("None")
                
                print(sys_output[document]["golden_clusters"][entity_ind])
                golden_cluster_pos = sys_output[document]["golden_clusters"][entity_ind].index(mention)
                if golden_cluster_pos == 0:
                    decision_dict["antecedent_dist_gt"].append(-1)
                else:
                    previous_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos-1]
                    decision_dict["antecedent_dist_gt"].append(mention[0] - previous_mention[0])
                
                ## Next mention distance
                if golden_cluster_pos == len(sys_output[document]["golden_clusters"][entity_ind]) - 1:
                    decision_dict["next_dist_gt"].append(-1)
                else:
                    next_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos+1]
                    decision_dict["next_dist_gt"].append(next_mention[0] - mention[0])
                
                ## Nearest mention distance
                if golden_cluster_pos == 0:
                    decision_dict["nearest_dist_gt"].append(next_mention[0] - mention[0])
                elif golden_cluster_pos == len(sys_output[document]["golden_clusters"][entity_ind]) - 1:
                    decision_dict["nearest_dist_gt"].append(mention[0] - previous_mention[0])
                else:
                    previous_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos-1]
                    next_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos+1]
                    decision_dict["nearest_dist_gt"].append(min(next_mention[0] - mention[0],mention[0] - previous_mention[0]))

                pred_cluster_entity = sys_output[document]["predicted_clusters"][entity_ind]
                decision_dict["antecedent_dist_pred"].append(find_previous_span_dist(pred_cluster_entity,mention))
                                
            else:
                decision_dict["major_mention"].append(False)
                decision_dict["mention_ind"].append(-1)
                decision_dict["entity_ind"].append(-1)
                decision_dict["entity_name"].append("None")
                decision_dict["category"].append("None")
                decision_dict["type"].append("None")

                decision_dict["antecedent_dist_gt"].append(None)
                decision_dict["next_dist_gt"].append(None)
                decision_dict["nearest_dist_gt"].append(None)
                decision_dict["antecedent_dist_pred"].append(None)
                decision_dict["mapped_antecedent_dist_gt"].append(None)
                            
            decision_dict["correct"].append(False)
            if decision_dict["major_mention"][-1]:
                decision_dict["correct_major"].append(False)
                decision_dict["detailed_error_type"].append("Missing Mention")
            else:
                decision_dict["correct_major"].append(True) ## It is correct in major becuase it failed in a mention that isn't golden
                decision_dict["detailed_error_type"].append("None")
            
            decision_dict["mapped_antecedent_dist_gt"].append(None)
            decision_dict["mapped_antecedent_dist_pred"].append(None)
            
                            
            ## Not adding score here because this is not a sota concept
    
    return decision_dict
    

In [ ]:
decision_dict = process_failed_mentions(sys_output, decision_dict)
for key in decision_dict:
    print(f"Length of {key}: ", len(decision_dict[key]))

Length of doc_key:  0
Length of broad_mention_type:  0
Length of nested_mention:  0
Length of mention_start:  0
Length of mention_end:  0
Length of mention_start_stbound:  0
Length of mention_end_stbound:  0
Length of mention_str:  0
Length of mapped_entity_ind:  0
Length of mapped_entity_name:  0
Length of golden_mention:  0
Length of major_mention:  0
Length of mention_ind:  0
Length of entity_ind:  0
Length of entity_name:  0
Length of category:  0
Length of type:  0
Length of antecedent_dist_gt:  0
Length of next_dist_gt:  0
Length of nearest_dist_gt:  0
Length of antecedent_dist_pred:  0
Length of correct:  0
Length of correct_major:  0
Length of detailed_error_type:  0
Length of mapped_antecedent_dist_gt:  0
Length of mapped_antecedent_dist_pred:  0


In [ ]:
def process_predicted_mentions(sys_output, decision_dict):
    for document in sys_output.keys():
        if len(sys_output[document]["predicted_clusters"]) < len(sys_output[document]["golden_clusters"]) + 1:
            predicted_clusters_proper = sys_output[document]["predicted_clusters"]
        else:
            predicted_clusters_proper = sys_output[document]["predicted_clusters"][:-1] ## Remove 
        predicted_mentions_proper = sorted([mention for cluster in predicted_clusters_proper for mention in cluster])
        # print(predicted_mentions_proper)
        major_entity_names = major_entities[document]["entity_name"] + ["Others"]
        # print(major_entity_names)
        for mention in predicted_mentions_proper:
            decision_dict["doc_key"].append(document)
            decision_dict["broad_mention_type"].append("Predicted Mention")
            mention_ind_original = sys_output[document]["predicted_mentions"].index(mention)
            decision_dict["nested_mention"].append(sys_output[document]["predicted_mentions_nested"][mention_ind_original])
            
            decision_dict["mention_start"].append(mention[0])
            decision_dict["mention_end"].append(mention[1])
            mention_index_in_predicted = sys_output[document]["predicted_mentions"].index(mention)
            decision_dict["mention_start_stbound"].append(sys_output[document]["predicted_mentions_stbound"][mention_index_in_predicted][0])
            decision_dict["mention_end_stbound"].append(sys_output[document]["predicted_mentions_stbound"][mention_index_in_predicted][1])
            decision_dict["mention_str"].append(get_mention_str_tbound(mention, docs_processed[document]))
            mapped_entity_ind = sys_output[document]["predicted_mentions_vs_clusters"][tuple(mention)]
            decision_dict["mapped_entity_ind"].append(mapped_entity_ind)
            decision_dict["mapped_entity_name"].append(major_entity_names[mapped_entity_ind])
            
            is_golden_mention = 1 if mention in sys_output[document]["gt_mentions"] else 0
            decision_dict["golden_mention"].append(is_golden_mention)
            if is_golden_mention:
                # print(original_index_list_key,docs_processed[document][original_index_list_key])
                entity_ind = sys_output[document]["gt_mentions_vs_clusters"][tuple(mention)]
                # print("Entity_Ind: ",entity_ind)
                if entity_ind < len(major_entity_names) - 1:
                    decision_dict["major_mention"].append(True)
                else:
                    decision_dict["major_mention"].append(False)
                mention_ind_gold_orig = docs_processed[document][original_index_list_key].index(mention)
                decision_dict["mention_ind"].append(mention_ind_gold_orig)
                decision_dict["entity_ind"].append(entity_ind)
                decision_dict["entity_name"].append(major_entity_names[entity_ind])
            
                if doc_tsv_addr is not None:
                    decision_dict["category"].append(docs_processed[document]["mentions_vs_mentionctgry"][mention_ind_gold_orig])
                    decision_dict["type"].append(docs_processed[document]["mentions_vs_mentiontype"][mention_ind_gold_orig])  
                else:
                    if decision_dict["mention_str"][-1].lower().strip() in PRONOUNS_GROUPS:
                        decision_dict["category"].append("PRON")
                    else:
                        decision_dict["category"].append("NOM")
                    decision_dict["type"].append("None")
                
                golden_cluster_pos = sys_output[document]["golden_clusters"][entity_ind].index(mention)
                if golden_cluster_pos == 0:
                    decision_dict["antecedent_dist_gt"].append(-1)
                else:
                    previous_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos-1]
                    decision_dict["antecedent_dist_gt"].append(mention[0] - previous_mention[0])

                ## Next mention distance
                if golden_cluster_pos == len(sys_output[document]["golden_clusters"][entity_ind]) - 1:
                    decision_dict["next_dist_gt"].append(-1)
                else:
                    next_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos+1]
                    decision_dict["next_dist_gt"].append(next_mention[0] - mention[0])
                
                ## Nearest mention distance
                if golden_cluster_pos == 0:
                    decision_dict["nearest_dist_gt"].append(next_mention[0] - mention[0])
                elif golden_cluster_pos == len(sys_output[document]["golden_clusters"][entity_ind]) - 1:
                    decision_dict["nearest_dist_gt"].append(mention[0] - previous_mention[0])
                else:
                    previous_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos-1]
                    next_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos+1]
                    decision_dict["nearest_dist_gt"].append(min(next_mention[0] - mention[0],mention[0] - previous_mention[0]))
                
                pred_cluster = sys_output[document]["predicted_clusters"][entity_ind]
                decision_dict["antecedent_dist_pred"].append(find_previous_span_dist(pred_cluster,mention))
            
            else:
                # correct = (mapped_entity_ind == len(major_entity_names) - 1)
                decision_dict["major_mention"].append(False)
                decision_dict["mention_ind"].append(-1)
                decision_dict["entity_ind"].append(-1)
                decision_dict["entity_name"].append("None")
                decision_dict["category"].append("None")
                decision_dict["type"].append("None")

                decision_dict["antecedent_dist_gt"].append(None)
                decision_dict["next_dist_gt"].append(None)
                decision_dict["nearest_dist_gt"].append(None)
                decision_dict["antecedent_dist_pred"].append(None)

            
            if is_golden_mention:
                correct = (mapped_entity_ind == entity_ind)
                decision_dict["correct"].append(correct)
                decision_dict["correct_major"].append(correct)
                if correct == False:
                    if mapped_entity_ind < len(major_entity_names) -1 and entity_ind < len(major_entity_names) -1:
                        decision_dict["detailed_error_type"].append("c_c")
                    elif mapped_entity_ind < len(major_entity_names) - 1 and entity_ind == len(major_entity_names) - 1:
                        decision_dict["detailed_error_type"].append("o_c")
                    else:
                        decision_dict["detailed_error_type"].append("c_o")
                else:
                    decision_dict["detailed_error_type"].append("None")
            else:
                correct = (mapped_entity_ind == len(major_entity_names) - 1)
                decision_dict["correct"].append(correct)
                decision_dict["correct_major"].append(correct)
                if correct == False:
                    decision_dict["detailed_error_type"].append("ngo_c")
                else:
                    decision_dict["detailed_error_type"].append("None")
            
            decision_dict["mapped_antecedent_dist_gt"].append(find_previous_span_dist(sys_output[document]["golden_clusters"][mapped_entity_ind],mention))
            pred_mapped_cluster = sys_output[document]["predicted_clusters"][mapped_entity_ind]
            decision_dict["mapped_antecedent_dist_pred"].append(find_previous_span_dist(pred_mapped_cluster,mention))

            if sys_output[document].get("coref_scores",None) is not None:
                decision_dict["score"].append(sys_output[document]["coref_scores"][mention_index_in_predicted])

    return decision_dict
    

In [ ]:
decision_dict = process_predicted_mentions(sys_output, decision_dict)
for key in decision_dict:
    print(f"Length of {key}: ", len(decision_dict[key]))

Length of doc_key:  2352
Length of broad_mention_type:  2352
Length of nested_mention:  2352
Length of mention_start:  2352
Length of mention_end:  2352
Length of mention_start_stbound:  2352
Length of mention_end_stbound:  2352
Length of mention_str:  2352
Length of mapped_entity_ind:  2352
Length of mapped_entity_name:  2352
Length of golden_mention:  2352
Length of major_mention:  2352
Length of mention_ind:  2352
Length of entity_ind:  2352
Length of entity_name:  2352
Length of category:  2352
Length of type:  2352
Length of antecedent_dist_gt:  2352
Length of next_dist_gt:  2352
Length of nearest_dist_gt:  2352
Length of antecedent_dist_pred:  2352
Length of correct:  2352
Length of correct_major:  2352
Length of detailed_error_type:  2352
Length of mapped_antecedent_dist_gt:  2352
Length of mapped_antecedent_dist_pred:  2352


In [ ]:
def process_missing_mentions(sys_output,decision_dict):
    for document in sys_output.keys():
        golden_clusters = sys_output[document]["golden_clusters"]
        golden_mentions = sys_output[document]["gt_mentions"]
        predicted_mentions = sys_output[document]["predicted_mentions"]
        # print(predicted_mentions_proper)
        major_entity_names = major_entities[document]["entity_name"] + ["Others"]
        # print(major_entity_names)
        for mention_ind,mention in enumerate(golden_mentions):
            if mention not in predicted_mentions:
                mention_ind_gold_orig = docs_processed[document][original_index_list_key].index(mention)
                decision_dict["doc_key"].append(document)
                decision_dict["broad_mention_type"].append("Missing Mention")
                decision_dict["nested_mention"].append(sys_output[document]["gt_mentions_nested"][mention_ind])
                
                decision_dict["mention_start"].append(mention[0])
                decision_dict["mention_end"].append(mention[1])
                decision_dict["mention_start_stbound"].append(sys_output[document]["golden_mentions_stbound"][mention_ind_gold_orig][0])
                decision_dict["mention_end_stbound"].append(sys_output[document]["golden_mentions_stbound"][mention_ind_gold_orig][1])
                decision_dict["mention_str"].append(get_mention_str_tbound(mention, docs_processed[document]))
                decision_dict["mapped_entity_ind"].append(-1)
                decision_dict["mapped_entity_name"].append("None")
                
                
                decision_dict["golden_mention"].append(1)
                if sys_output[document]["gt_mentions_vs_clusters"][tuple(mention)] < len(major_entity_names) - 1:
                    major_mention = 1
                else:
                    major_mention = 0
                decision_dict["major_mention"].append((major_mention == 1))
                decision_dict["mention_ind"].append(mention_ind_gold_orig)
                entity_ind = sys_output[document]["gt_mentions_vs_clusters"][tuple(mention)]
                decision_dict["entity_ind"].append(entity_ind)
                decision_dict["entity_name"].append(major_entity_names[entity_ind])
                
                if doc_tsv_addr is not None:
                    decision_dict["category"].append(docs_processed[document]["mentions_vs_mentionctgry"][mention_ind_gold_orig])
                    decision_dict["type"].append(docs_processed[document]["mentions_vs_mentiontype"][mention_ind_gold_orig])
                else:
                    if decision_dict["mention_str"][-1].lower().strip() in PRONOUNS_GROUPS:
                        decision_dict["category"].append("PRON")
                    else:
                        decision_dict["category"].append("NOM")
                    decision_dict["type"].append("None")
                
                golden_cluster_pos = sys_output[document]["golden_clusters"][entity_ind].index(mention)
                if golden_cluster_pos == 0:
                    decision_dict["antecedent_dist_gt"].append(-1)
                else:
                    previous_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos-1]
                    decision_dict["antecedent_dist_gt"].append(mention[0] - previous_mention[0])

                ## Next mention distance
                if golden_cluster_pos == len(sys_output[document]["golden_clusters"][entity_ind]) - 1:
                    decision_dict["next_dist_gt"].append(-1)
                else:
                    next_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos+1]
                    decision_dict["next_dist_gt"].append(next_mention[0] - mention[0])
                
                ## Nearest mention distance
                if golden_cluster_pos == 0:
                    decision_dict["nearest_dist_gt"].append(next_mention[0] - mention[0])
                elif golden_cluster_pos == len(sys_output[document]["golden_clusters"][entity_ind]) - 1:
                    decision_dict["nearest_dist_gt"].append(mention[0] - previous_mention[0])
                else:
                    previous_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos-1]
                    next_mention = sys_output[document]["golden_clusters"][entity_ind][golden_cluster_pos+1]
                    decision_dict["nearest_dist_gt"].append(min(next_mention[0] - mention[0],mention[0] - previous_mention[0]))
                
                pred_cluster_entity = sys_output[document]["predicted_clusters"][entity_ind]
                decision_dict["antecedent_dist_pred"].append(find_previous_span_dist(pred_cluster_entity,mention))


                decision_dict["correct"].append(False)
                if major_mention:
                    decision_dict["correct_major"].append(False)
                    decision_dict["detailed_error_type"].append("Missing Mention")
                else:
                    decision_dict["correct_major"].append(True)
                    decision_dict["detailed_error_type"].append("None")
                 
                
                
                decision_dict["mapped_antecedent_dist_gt"].append(None)
                decision_dict["mapped_antecedent_dist_pred"].append(None)
                
                if sys_output[document].get("coref_scores",None) is not None:
                    decision_dict["score"].append(-99999)
    return decision_dict
    

In [ ]:
decision_dict = process_missing_mentions(sys_output, decision_dict)
for key in decision_dict:
    print(f"Length of {key}: ", len(decision_dict[key]))

Length of doc_key:  5924
Length of broad_mention_type:  5924
Length of nested_mention:  5924
Length of mention_start:  5924
Length of mention_end:  5924
Length of mention_start_stbound:  5924
Length of mention_end_stbound:  5924
Length of mention_str:  5924
Length of mapped_entity_ind:  5924
Length of mapped_entity_name:  5924
Length of golden_mention:  5924
Length of major_mention:  5924
Length of mention_ind:  5924
Length of entity_ind:  5924
Length of entity_name:  5924
Length of category:  5924
Length of type:  5924
Length of antecedent_dist_gt:  5924
Length of next_dist_gt:  5924
Length of nearest_dist_gt:  5924
Length of antecedent_dist_pred:  5924
Length of correct:  5924
Length of correct_major:  5924
Length of detailed_error_type:  5924
Length of mapped_antecedent_dist_gt:  5924
Length of mapped_antecedent_dist_pred:  5924


In [ ]:
decision_df = pd.DataFrame(decision_dict)
d = dtale.show(decision_df,subprocess=False, host='localhost',enable_custom_filters=True)

2024-07-12 19:40:45,786 - WARNING  - Custom filtering enabled. Custom filters are vulnerable to code injection attacks, please only use in trusted environments.
2024-07-12 19:40:45,788 - INFO     - D-Tale started at: http://localhost:40001


2024-07-12 20:51:00,356 - INFO     - Executing shutdown due to inactivity...
2024-07-12 20:51:00,361 - INFO     - Executing shutdown...
2024-07-12 20:51:00,362 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
/home/admin/miniconda3/envs/myenv/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:359: UserWarning:

torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead

